**Import**

In [1]:
import torch
print('pytorch version: {}'.format(torch.__version__))

pytorch version: 2.1.2


In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 2.1.2
GPU 사용 가능 여부: True


In [5]:
# 이걸 해줘야 matplotlib 시행 시 에러가 안 남
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

**Setting hyperparameters**

In [6]:
batch_size = 64
num_epochs = 30
learning_rate = 0.001

**Data loader**

In [7]:
#data_dir = '../data/OCT'  # 압축 해제된 데이터셋의 디렉토리 경로

# For Kaggle notebook
data_dir = '/kaggle/input/kermany2018/OCT2017 /'

In [9]:
class OCTDataset(Dataset):
    def __init__(self, data_dir, mode, transform=None):
        self.all_data = sorted(glob.glob(os.path.join(data_dir, mode,'*', '*')))
        self.transform = transform
    
    def __getitem__(self, index):
        data_path = self.all_data[index]
        img = Image.open(data_path)
        if self.transform is not None:
            img = self.transform(img)
        img = img.squeeze()
        img = torch.stack((img, img, img,), dim=0)  # VGG16이 3 channel이라 concat시키기
        name = os.path.basename(data_path)
        if name.startswith('NORMAL'):
            label = 0
        elif name.startswith('CNV'):
            label = 1
        elif name.startswith('DME'):
            label = 2
        elif name.startswith('DRUSEN'):
            label = 3
        return img, label
    
    def __len__(self):
        length = len(self.all_data)
        return length

In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(0.1881,0.1850)
    ]),
    'val': transforms.Compose([
        transforms.Resize([256]),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(0.1881,0.1850)
    ])
}

In [11]:
train_data = OCTDataset(data_dir=data_dir, mode='train', transform=data_transforms['train'])
val_data = OCTDataset(data_dir=data_dir, mode='val', transform=data_transforms['val'])
test_data = OCTDataset(data_dir=data_dir, mode='test', transform=data_transforms['val'])

In [12]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=2)  # num_worker로 Multi Process Data Loading 구현
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2) # Kaggle CPU = 2 core
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)

In [13]:
train_mean = 0.1881
train_std = 0.1850

**ResNet50 Transfer Learning**

In [14]:
class TransferResNet(nn.Module):
    def __init__ (self):
        super(TransferResNet, self).__init__()
        self.ResNet50 = torchvision.models.resnet50(pretrained=True)
        self.ResNet50.fc = nn.Linear(2048, 4)        # Final layer input = 512, Final layer output = 4 (num of classes) (original: 1000)
    
    def forward(self,x):
        return self.ResNet50(x)

In [15]:
resnet50_transfer = TransferResNet().to(device)
resnet50_transfer

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 155MB/s] 


TransferResNet(
  (ResNet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
  

In [16]:
count = 0
for name, param in resnet50_transfer.named_parameters():
    count += 1
    print(f"count:{count},",name, param.requires_grad)

count:1, ResNet50.conv1.weight True
count:2, ResNet50.bn1.weight True
count:3, ResNet50.bn1.bias True
count:4, ResNet50.layer1.0.conv1.weight True
count:5, ResNet50.layer1.0.bn1.weight True
count:6, ResNet50.layer1.0.bn1.bias True
count:7, ResNet50.layer1.0.conv2.weight True
count:8, ResNet50.layer1.0.bn2.weight True
count:9, ResNet50.layer1.0.bn2.bias True
count:10, ResNet50.layer1.0.conv3.weight True
count:11, ResNet50.layer1.0.bn3.weight True
count:12, ResNet50.layer1.0.bn3.bias True
count:13, ResNet50.layer1.0.downsample.0.weight True
count:14, ResNet50.layer1.0.downsample.1.weight True
count:15, ResNet50.layer1.0.downsample.1.bias True
count:16, ResNet50.layer1.1.conv1.weight True
count:17, ResNet50.layer1.1.bn1.weight True
count:18, ResNet50.layer1.1.bn1.bias True
count:19, ResNet50.layer1.1.conv2.weight True
count:20, ResNet50.layer1.1.bn2.weight True
count:21, ResNet50.layer1.1.bn2.bias True
count:22, ResNet50.layer1.1.conv3.weight True
count:23, ResNet50.layer1.1.bn3.weight Tr

In [17]:
# Layer 4의 required_grad True로 바꾸기 (Layer 4의 시작: 130) -> 130 이전은 false, 이후는 True
count = 0
for param in resnet50_transfer.ResNet50.parameters():
    count += 1
    if count >= 130:
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in resnet50_transfer.named_parameters():
    print(name, param.requires_grad)

ResNet50.conv1.weight False
ResNet50.bn1.weight False
ResNet50.bn1.bias False
ResNet50.layer1.0.conv1.weight False
ResNet50.layer1.0.bn1.weight False
ResNet50.layer1.0.bn1.bias False
ResNet50.layer1.0.conv2.weight False
ResNet50.layer1.0.bn2.weight False
ResNet50.layer1.0.bn2.bias False
ResNet50.layer1.0.conv3.weight False
ResNet50.layer1.0.bn3.weight False
ResNet50.layer1.0.bn3.bias False
ResNet50.layer1.0.downsample.0.weight False
ResNet50.layer1.0.downsample.1.weight False
ResNet50.layer1.0.downsample.1.bias False
ResNet50.layer1.1.conv1.weight False
ResNet50.layer1.1.bn1.weight False
ResNet50.layer1.1.bn1.bias False
ResNet50.layer1.1.conv2.weight False
ResNet50.layer1.1.bn2.weight False
ResNet50.layer1.1.bn2.bias False
ResNet50.layer1.1.conv3.weight False
ResNet50.layer1.1.bn3.weight False
ResNet50.layer1.1.bn3.bias False
ResNet50.layer1.2.conv1.weight False
ResNet50.layer1.2.bn1.weight False
ResNet50.layer1.2.bn1.bias False
ResNet50.layer1.2.conv2.weight False
ResNet50.layer1.2.bn

In [18]:
# Layer 4의 Conv layer를 다시 initialization
for name, layer in resnet50_transfer.ResNet50.named_children():
    if name == 'layer4':
        bottleneck_index = 0
        conv_index = 1
        for name, param in resnet50_transfer.ResNet50.named_parameters():
            # Layer 4의 Conv layer parameter initialization
            if name == 'layer4.'+str(bottleneck_index)+'.conv'+str(conv_index)+'.weight':
                print('layer4의 '+str(bottleneck_index)+'번째 bottleneck의 conv'+str(conv_index)+'.weight')
                nn.init.xavier_uniform_(param)
                print(name+'의 conv filter initialization setting 완료')
                print()
                conv_index += 1
                if name == 'layer4.'+str(bottleneck_index)+'.conv3.weight':
                    bottleneck_index += 1
                    conv_index = 1

layer4의 0번째 bottleneck의 conv1.weight
layer4.0.conv1.weight의 conv filter initialization setting 완료

layer4의 0번째 bottleneck의 conv2.weight
layer4.0.conv2.weight의 conv filter initialization setting 완료

layer4의 0번째 bottleneck의 conv3.weight
layer4.0.conv3.weight의 conv filter initialization setting 완료

layer4의 1번째 bottleneck의 conv1.weight
layer4.1.conv1.weight의 conv filter initialization setting 완료

layer4의 1번째 bottleneck의 conv2.weight
layer4.1.conv2.weight의 conv filter initialization setting 완료

layer4의 1번째 bottleneck의 conv3.weight
layer4.1.conv3.weight의 conv filter initialization setting 완료

layer4의 2번째 bottleneck의 conv1.weight
layer4.2.conv1.weight의 conv filter initialization setting 완료

layer4의 2번째 bottleneck의 conv2.weight
layer4.2.conv2.weight의 conv filter initialization setting 완료

layer4의 2번째 bottleneck의 conv3.weight
layer4.2.conv3.weight의 conv filter initialization setting 완료



In [19]:
# Layer 4의 batch normalization gamma, beta initialization
# 전부 initialization시키므로 처음 시작을 0, 1로 시작, 만약 좀 더 뒤로 바꾸고 싶으면 초기 시작 값을 바꾸기
bottleneck_index = 0
bn_index = 1
for name, param in resnet50_transfer.ResNet50.named_parameters():          
    if name == 'layer4.'+str(bottleneck_index)+'.'+'bn1.weight':
        print('layer4의 '+str(bottleneck_index)+'번째 bottleneck의 '+'bn1.weight')
        nn.init.ones_(param)
        print(name+'의 gamma one setting 완료')
        print()
    elif name == 'layer4.'+str(bottleneck_index)+'.'+'bn2.weight':
        print('layer4의 '+str(bottleneck_index)+'번째 bottleneck의 '+'bn2.weight')
        nn.init.ones_(param)
        print(name+'의 gamma one setting 완료')
        print()
    elif name == 'layer4.'+str(bottleneck_index)+'.'+'bn3.weight':
        print('layer4의 '+str(bottleneck_index)+'번째 bottleneck의 '+'bn3.weight')
        nn.init.zeros_(param)
        print(name+'의 gamma zero setting 완료')    # residual block 마지막의 batchnorm의 gamma는 0으로, 나머지는 1로 초기화시킨다고 함
        print()
    elif name == 'layer4.'+str(bottleneck_index)+'.'+'bn'+str(bn_index)+'.bias':
        print('layer4.'+str(bottleneck_index)+'.'+'bn'+str(bn_index)+'.bias')     # bias는 0으로 초기화
        nn.init.zeros_(param)
        bn_index += 1
        print(name+'의 beta zero setting 완료')
        if bn_index == 4:
            bn_index=1
            bottleneck_index += 1
        print()

layer4의 0번째 bottleneck의 bn1.weight
layer4.0.bn1.weight의 gamma one setting 완료

layer4.0.bn1.bias
layer4.0.bn1.bias의 beta zero setting 완료

layer4의 0번째 bottleneck의 bn2.weight
layer4.0.bn2.weight의 gamma one setting 완료

layer4.0.bn2.bias
layer4.0.bn2.bias의 beta zero setting 완료

layer4의 0번째 bottleneck의 bn3.weight
layer4.0.bn3.weight의 gamma zero setting 완료

layer4.0.bn3.bias
layer4.0.bn3.bias의 beta zero setting 완료

layer4의 1번째 bottleneck의 bn1.weight
layer4.1.bn1.weight의 gamma one setting 완료

layer4.1.bn1.bias
layer4.1.bn1.bias의 beta zero setting 완료

layer4의 1번째 bottleneck의 bn2.weight
layer4.1.bn2.weight의 gamma one setting 완료

layer4.1.bn2.bias
layer4.1.bn2.bias의 beta zero setting 완료

layer4의 1번째 bottleneck의 bn3.weight
layer4.1.bn3.weight의 gamma zero setting 완료

layer4.1.bn3.bias
layer4.1.bn3.bias의 beta zero setting 완료

layer4의 2번째 bottleneck의 bn1.weight
layer4.2.bn1.weight의 gamma one setting 완료

layer4.2.bn1.bias
layer4.2.bn1.bias의 beta zero setting 완료

layer4의 2번째 bottleneck의 bn2.weight
laye

**Saving models, Training, Validation, Test**

In [20]:
def save_model(model, epoch, optimizer, epoch_loss, val_accuracy, saved_dir):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss
    }
    val_accuracy = round(val_accuracy,1)
    file_name = str(f"ckpoint_model_{epoch+1}_{val_accuracy}%.pt")
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

In [21]:
logs_base_dir = "./logs/logs_transfer"
os.makedirs(logs_base_dir, exist_ok=True)

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(logs_base_dir)

2024-05-03 05:51:41.721955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 05:51:41.722083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 05:51:41.861992: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [22]:
def train(num_epochs, model, data_loader, criterion, optimizer, scheduler, val_every, device, saved_dir, writer):
    print('Start training..')
    torch.cuda.empty_cache()
    best_loss = 9999999
    train_loss_list = []
    val_loss_list = []
    train_accuracy_list = []
    val_accuracy_list = []
    for epoch in tqdm(range(num_epochs), desc='epoch'):
        count = 0.0
        # running_loss = 0.0
        for i, (imgs, labels) in tqdm(enumerate(data_loader), desc="in epoch"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Saving train loss
            train_loss_list.append(loss.item())
            writer.add_scalar('Loss/Train', loss, i)

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            # Saving train accuracy
            train_accuracy_list.append(accuracy)
            writer.add_scalar('Accuracy/Train', accuracy, i)

            if (i+1) % 30 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100))
            #if i % 30 == 29:
                #loss_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
                #running_loss = 0.0
        if (epoch + 1) % val_every == 0:
            avrg_loss, val_accuracy = validation(epoch + 1, model, val_loader, criterion, device)
            # Saving validation loss and accuracy
            val_loss_list.append(avrg_loss)
            writer.add_scalar('Loss/Validation', avrg_loss, epoch+1)
            val_accuracy_list.append(val_accuracy)
            writer.add_scalar('Accuracy/Validation', val_accuracy, epoch+1)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, epoch, optimizer, avrg_loss, val_accuracy, saved_dir)
            else:
                count += 1
                if count >= 10:
                    print('Best performance does not occur within 10 epochs. Early stopping!!')
                    scheduler.step()
                    writer.flush()
                    return train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list
            ''' Only for local setting
            loss_tracker(loss_plt, torch.Tensor([avrg_loss]), torch.Tensor([epoch]))'''
    scheduler.step()
    writer.flush()
    return train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list

In [23]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs,labels)
            
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        val_accuracy = correct / total * 100
        print('Validation #{}  Accuracy: {:.2f}%  Average Loss: {:.4f}'.format(epoch, val_accuracy, avrg_loss))
    model.train()
    return avrg_loss, val_accuracy

In [24]:
def test(model, data_loader, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        answer_list = []
        pred_list = []
        pred_proba_list = []
        
        for i, (imgs, labels) in tqdm(enumerate(data_loader)):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)

            pred_probability = F.softmax(outputs, dim=1)
            pred_proba_list.append(pred_probability.tolist())
            
            _, argmax = torch.max(outputs, 1)
            pred_list.append(argmax.tolist())

            total += imgs.size(0)
            correct += (labels == argmax).sum().item()
            
            answer_list.append(labels.tolist())
        
        print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))
    model.train()        
    return pred_list, answer_list, pred_proba_list

**Training**

In [25]:
torch.manual_seed(7777)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet50_transfer.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.00001)

val_every = 1
saved_dir = './saved'

In [26]:
train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list = train(num_epochs, resnet50_transfer, train_loader, criterion, optimizer, scheduler, val_every, device, saved_dir, writer)

Start training..


epoch:   0%|          | 0/30 [00:00<?, ?it/s]

in epoch: 0it [00:00, ?it/s]

Epoch [1/30], Step [30/1305], Loss: 0.5368, Accuracy: 78.12%
Epoch [1/30], Step [60/1305], Loss: 0.2252, Accuracy: 89.06%
Epoch [1/30], Step [90/1305], Loss: 0.4083, Accuracy: 87.50%
Epoch [1/30], Step [120/1305], Loss: 0.5219, Accuracy: 79.69%
Epoch [1/30], Step [150/1305], Loss: 0.3057, Accuracy: 89.06%
Epoch [1/30], Step [180/1305], Loss: 0.2546, Accuracy: 89.06%
Epoch [1/30], Step [210/1305], Loss: 0.2323, Accuracy: 93.75%
Epoch [1/30], Step [240/1305], Loss: 0.2057, Accuracy: 90.62%
Epoch [1/30], Step [270/1305], Loss: 0.1475, Accuracy: 95.31%
Epoch [1/30], Step [300/1305], Loss: 0.1462, Accuracy: 96.88%
Epoch [1/30], Step [330/1305], Loss: 0.2423, Accuracy: 87.50%
Epoch [1/30], Step [360/1305], Loss: 0.1811, Accuracy: 92.19%
Epoch [1/30], Step [390/1305], Loss: 0.1088, Accuracy: 95.31%
Epoch [1/30], Step [420/1305], Loss: 0.0902, Accuracy: 96.88%
Epoch [1/30], Step [450/1305], Loss: 0.2736, Accuracy: 90.62%
Epoch [1/30], Step [480/1305], Loss: 0.1463, Accuracy: 93.75%
Epoch [1/30

in epoch: 0it [00:00, ?it/s]

...

Epoch [28/30], Step [30/1305], Loss: 0.0444, Accuracy: 98.44%
Epoch [28/30], Step [60/1305], Loss: 0.0190, Accuracy: 98.44%
Epoch [28/30], Step [90/1305], Loss: 0.0293, Accuracy: 98.44%
Epoch [28/30], Step [120/1305], Loss: 0.0277, Accuracy: 100.00%
Epoch [28/30], Step [150/1305], Loss: 0.0245, Accuracy: 98.44%
Epoch [28/30], Step [180/1305], Loss: 0.1550, Accuracy: 95.31%
Epoch [28/30], Step [210/1305], Loss: 0.0063, Accuracy: 100.00%
Epoch [28/30], Step [240/1305], Loss: 0.0152, Accuracy: 100.00%
Epoch [28/30], Step [270/1305], Loss: 0.0034, Accuracy: 100.00%
Epoch [28/30], Step [300/1305], Loss: 0.1883, Accuracy: 93.75%
Epoch [28/30], Step [330/1305], Loss: 0.1089, Accuracy: 95.31%
Epoch [28/30], Step [360/1305], Loss: 0.0298, Accuracy: 98.44%
Epoch [28/30], Step [390/1305], Loss: 0.0537, Accuracy: 96.88%
Epoch [28/30], Step [420/1305], Loss: 0.0016, Accuracy: 100.00%
Epoch [28/30], Step [450/1305], Loss: 0.0209, Accuracy: 100.00%
Epoch [28/30], Step [480/1305], Loss: 0.0114, Accura

in epoch: 0it [00:00, ?it/s]

Epoch [29/30], Step [30/1305], Loss: 0.0080, Accuracy: 100.00%
Epoch [29/30], Step [60/1305], Loss: 0.1101, Accuracy: 95.31%
Epoch [29/30], Step [90/1305], Loss: 0.0133, Accuracy: 100.00%
Epoch [29/30], Step [120/1305], Loss: 0.0449, Accuracy: 98.44%
Epoch [29/30], Step [150/1305], Loss: 0.0390, Accuracy: 98.44%
Epoch [29/30], Step [180/1305], Loss: 0.0157, Accuracy: 98.44%
Epoch [29/30], Step [210/1305], Loss: 0.0358, Accuracy: 98.44%
Epoch [29/30], Step [240/1305], Loss: 0.0471, Accuracy: 96.88%
Epoch [29/30], Step [270/1305], Loss: 0.0209, Accuracy: 100.00%
Epoch [29/30], Step [300/1305], Loss: 0.0056, Accuracy: 100.00%
Epoch [29/30], Step [330/1305], Loss: 0.0463, Accuracy: 98.44%
Epoch [29/30], Step [360/1305], Loss: 0.0151, Accuracy: 98.44%
Epoch [29/30], Step [390/1305], Loss: 0.0091, Accuracy: 100.00%
Epoch [29/30], Step [420/1305], Loss: 0.0671, Accuracy: 96.88%
Epoch [29/30], Step [450/1305], Loss: 0.0498, Accuracy: 96.88%
Epoch [29/30], Step [480/1305], Loss: 0.0080, Accurac

in epoch: 0it [00:00, ?it/s]

Epoch [30/30], Step [30/1305], Loss: 0.0137, Accuracy: 100.00%
Epoch [30/30], Step [60/1305], Loss: 0.0281, Accuracy: 100.00%
Epoch [30/30], Step [90/1305], Loss: 0.0606, Accuracy: 96.88%
Epoch [30/30], Step [120/1305], Loss: 0.0143, Accuracy: 100.00%
Epoch [30/30], Step [150/1305], Loss: 0.0170, Accuracy: 100.00%
Epoch [30/30], Step [180/1305], Loss: 0.0148, Accuracy: 100.00%
Epoch [30/30], Step [210/1305], Loss: 0.0441, Accuracy: 98.44%
Epoch [30/30], Step [240/1305], Loss: 0.0062, Accuracy: 100.00%
Epoch [30/30], Step [270/1305], Loss: 0.0958, Accuracy: 96.88%
Epoch [30/30], Step [300/1305], Loss: 0.0052, Accuracy: 100.00%
Epoch [30/30], Step [330/1305], Loss: 0.0032, Accuracy: 100.00%
Epoch [30/30], Step [360/1305], Loss: 0.0171, Accuracy: 98.44%
Epoch [30/30], Step [390/1305], Loss: 0.1596, Accuracy: 95.31%
Epoch [30/30], Step [420/1305], Loss: 0.0202, Accuracy: 100.00%
Epoch [30/30], Step [450/1305], Loss: 0.0197, Accuracy: 100.00%
Epoch [30/30], Step [480/1305], Loss: 0.0999, Ac

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.ylabel('Loss')
plt.plot(train_loss_list, label = 'Train')
plt.plot(val_loss_list, label = 'Validation')
plt.subplot(1,2,2)
plt.ylabel('Accuracy')
plt.plot(train_accuracy_list, label = 'Train')
plt.plot(val_accuracy_list, label = 'Validation')
# plt.savefig('graph.png',facecolor = 'w')
plt.show()

In [24]:
%load_ext tensorboard
%tensorboard --logdir='./logs/logs_transfer'

**Zip files**

In [27]:
import shutil
from IPython.display import FileLink
shutil.make_archive('OUTPUT', 'zip', '/kaggle/working/')
FileLink(r'OUTPUT.zip') # 여기에 zip파일 이름 넣기

/kaggle/working/OUTPUT.zip

In [ ]:
from IPython.display import FileLink
FileLink(r'OUTPUT.zip') # 여기에 zip파일 이름 넣기

In [29]:
import shutil
from IPython.display import FileLink
shutil.make_archive('OUTPUT1', 'zip', '/kaggle/working/')
FileLink(r'OUTPUT1.zip') # 여기에 zip파일 이름 넣기

KeyboardInterrupt: 

In [30]:
from IPython.display import FileLink
FileLink(r'OUTPUT1.zip') # 여기에 zip파일 이름 넣기

/kaggle/working/OUTPUT1.zip

In [ ]:
import shutil
shutil.make_archive('OUTPUT2', 'zip', '/kaggle/working/')

In [36]:
from IPython.display import FileLink
FileLink(r'OUTPUT2.zip') # 여기에 zip파일 이름 넣기

/kaggle/working/OUTPUT2.zip

In [37]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/saved/ckpoint_model_30_100.0%.pt') # 여기에 zip파일 이름 넣기

/kaggle/working/saved/ckpoint_model_30_100.0%.pt

In [38]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/saved/ckpoint_model_20_100.0%.pt') # 여기에 zip파일 이름 넣기

/kaggle/working/saved/ckpoint_model_20_100.0%.pt